In [23]:
from Utils import *
import pandas as pd
import glob
import configparser
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import anderson
from scipy.stats import f_oneway, kruskal
import json as js
import multiprocessing
import argparse
# __ns3_path = os.popen('locate "ns-3.41" | grep /ns-3.41$').read().splitlines()[0]
__ns3_path = "/media/experiments/ns-allinone-3.41/ns-3.41"
errorRate = 0.05
difference = 1.30
# sample_rate = 0.30
sample_rates = [0.5]
confidenceValue = 1.96 # 95% confidence interval
propagationDelay = 50000


In [24]:
def plot_box_plot(data, key, check, results_dir, results_dir_file, x, y_label):
    fig = plt.figure()
    fig.set_size_inches(11, 7)
    ax1 = fig.add_subplot(111)

    bp = ax1.boxplot(data)
    for box in bp['boxes']:
        box.set(linewidth=2)
    for whisker in bp['whiskers']:
        whisker.set_linewidth(2)
    for cap in bp['caps']:
        cap.set_linewidth(2)
    for median in bp['medians']:
        median.set_linewidth(2)

    ax1.set_xlabel('Sample Rate(#n/sec)', fontsize=25, labelpad=-1)
    ax1.set_ylabel(y_label, fontsize=25, labelpad=-1)
    ax1.tick_params(axis='y', labelsize=20)
    ax1.tick_params(axis='x', labelsize=20)
    ax1.set_xticklabels(x)
    # set the ylim to the max of the data + 10% and 0
    ax1.set_ylim(0, max([i for j in range(len(data)) for i in data[j]]) * 1.1)

    plt.title(key, fontsize=25)
    plt.setp(ax1.spines.values(), lw=1.5, color='black')
    plt.savefig('../Results/results_{}/{}_{}_{}_boxPLot.pdf'.format(results_dir, results_dir_file, check, key))
    plt.clf()
    plt.close()

In [25]:
def read_data_flowIndicator(results_dict):
    return list(results_dict['MaxEpsilonIneqDelay'].keys())

def read_queues_indicators(results_dict):
    queues = []
    for key in results_dict.keys():
        if ('Delaystd' in key):
            queues.append(key[:-8])
    return queues

def prepare_results(flows, e2e_sample_rates, queues_names):
    rounds_results = {}
    num_of_agg_switches = 2

    rounds_results['MaxEpsilonIneqDelay'] = {}
    rounds_results['MaxEpsilonIneqSuccessProb'] = {}
    rounds_results['MaxEpsilonIneqSuccessProb']['E2E_eventAvg'] = {}
    rounds_results['MaxEpsilonIneqSuccessProb']['sentTime_est'] = {}
    rounds_results['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'] = {}
    for sample_rate in e2e_sample_rates:
        rounds_results['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'][sample_rate] = {}
    rounds_results['errors'] = {}
    rounds_results['maxEpsilonDelay'] = {}
    rounds_results['DropRate'] = []


    for flow in flows:
        rounds_results['MaxEpsilonIneqDelay'][flow] = {}
        rounds_results['MaxEpsilonIneqSuccessProb']['E2E_eventAvg'][flow] = {}
        rounds_results['MaxEpsilonIneqSuccessProb']['sentTime_est'][flow] = {}
        for sample_rate in e2e_sample_rates:
            rounds_results['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'][sample_rate][flow] = {}
        rounds_results['errors'][flow] = {}
        rounds_results['maxEpsilonDelay'][flow] = {}
        for i in range(num_of_agg_switches):
            rounds_results['MaxEpsilonIneqDelay'][flow]['A' + str(i)] = 0
            rounds_results['MaxEpsilonIneqSuccessProb']['E2E_eventAvg'][flow]['A' + str(i)] = 0
            rounds_results['MaxEpsilonIneqSuccessProb']['sentTime_est'][flow]['A' + str(i)] = 0
            for sample_rate in e2e_sample_rates:
                rounds_results['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'][sample_rate][flow]['A' + str(i)] = 0
            rounds_results['errors'][flow]['A' + str(i)] =[]
            rounds_results['maxEpsilonDelay'][flow]['A' + str(i)] = []
    rounds_results['experiments'] = 0
    for queue in queues_names:
        rounds_results[queue + 'Delaystd'] = []
        rounds_results[queue + 'DelayMean'] = []
    return rounds_results

In [26]:
args_dir = 'params'
results_dir = None
if results_dir is not None:
    results_dir = args_dir + '_' + results_dir
else:
    results_dir = args_dir
# check = 'delay' # 'delay' or 'loss'
check = 'loss'

In [27]:
config = configparser.ConfigParser()
config.read('../Parameters.config')
experiments = int(config.get('Settings', 'experiments'))
if 'forward' in args_dir:
    rateScales = [float(x) for x in config.get('Settings', 'serviceRateScales').split(',')]
if 'reverse' in args_dir:
    rateScales = [float(x) for x in config.get('Settings', 'errorRateScale').split(',')]
if 'params' in args_dir:
    rateScales = [float(x) for x in config.get('Settings', 'sampleRateScales').split(',')]
    
results = {}
for rate in rateScales:
    for file in os.listdir('../Results/results_' + results_dir + '/'+str(rate)+'/'):
        if str(experiments) not in file:
            continue
        temp = {}
        with open('../Results/results_' + results_dir + '/'+str(rate)+'/'+file) as f:
            temp = js.load(f)
        
        flows = read_data_flowIndicator(temp)
        queues_names = read_queues_indicators(temp)

        results[rate] = prepare_results(flows, temp['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'].keys() if 'poisson_sentTime_est' in temp['MaxEpsilonIneqSuccessProb'] else [], queues_names)
        e2e_sample_rates = temp['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'].keys()
        x = round(6 * 300 / (rate * 2 * 945), 3)
        dropRate = 0
        for flow in flows:
            for i in range(2):
                results[rate]['MaxEpsilonIneqDelay'][flow]['A' + str(i)] = temp['MaxEpsilonIneqDelay'][flow]['A' + str(i)] / temp['experiments'] * 100
                
                results[rate]['MaxEpsilonIneqSuccessProb']['E2E_eventAvg'][flow]['A' + str(i)] = temp['MaxEpsilonIneqSuccessProb']['E2E_eventAvg'][flow]['A' + str(i)] / temp['experiments'] * 100
                results[rate]['MaxEpsilonIneqSuccessProb']['sentTime_est'][flow]['A' + str(i)] = temp['MaxEpsilonIneqSuccessProb']['sentTime_est'][flow]['A' + str(i)] / temp['experiments'] * 100
                for sample_rate in e2e_sample_rates:
                    results[rate]['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'][sample_rate][flow]['A' + str(i)] = temp['MaxEpsilonIneqSuccessProb']['poisson_sentTime_est'][sample_rate][flow]['A' + str(i)] / temp['experiments'] * 100
                results[rate]['errors'][flow]['A' + str(i)] = temp['errors'][flow]['A' + str(i)]
                results[rate]['maxEpsilonDelay'][flow]['A' + str(i)] = temp['maxEpsilonDelay'][flow]['A' + str(i)]
            for queue in queues_names:
                results[rate][queue + 'Delaystd'] = temp[queue + 'Delaystd']
                results[rate][queue + 'DelayMean'] = temp[queue + 'DelayMean']

            dropRate += (1.0 - np.mean(temp['EndToEndSuccessProb']['E2E_eventAvg'][flow]['A0']) + 1.0 - np.mean(temp['EndToEndSuccessProb']['E2E_eventAvg'][flow]['A1'])) / 2
        results[rate]['experiments'] = temp['experiments']

        results[rate]['DropRate'] = dropRate / len(flows)

In [28]:
check = 'loss'
# key of the first dictionary
for key in results[rateScales[0]]['MaxEpsilonIneqSuccessProb'].keys():
    if key != 'poisson_sentTime_est':
        data = [[] for i in range(len(rateScales))]
        for flow in flows:
            for i in range(len(rateScales)):
                data[i].append(max(results[rateScales[i]]['MaxEpsilonIneqSuccessProb'][key][flow]['A0'], results[rateScales[i]]['MaxEpsilonIneqSuccessProb'][key][flow]['A1']))
        plot_box_plot(data, 'loss based on ' + key, check, results_dir, args_dir, [int(x * 4500) for x in rateScales], "Success Rate (%)")
    if key == 'poisson_sentTime_est':
        for sample_rate in sample_rates:
            data = [[] for i in range(len(rateScales))]
            for flow in flows:
                for i in range(len(rateScales)):
                    data[i].append(max(results[rateScales[i]]['MaxEpsilonIneqSuccessProb'][key][str(sample_rate)][flow]['A0'], results[rateScales[i]]['MaxEpsilonIneqSuccessProb'][key][str(sample_rate)][flow]['A1']))
            plot_box_plot(data, 'loss based on ' + key, check, results_dir, args_dir, [int(x * 4500) for x in rateScales], "Success Rate (%)")
    
# plot for forward experiment -- BoxPLots(delay): 
check = 'delay'
data = [[] for i in range(len(rateScales))]
for flow in flows:
    for i in range(len(rateScales)):
        data[i].append(max(results[rateScales[i]]['MaxEpsilonIneqDelay'][flow]['A0'], results[rateScales[i]]['MaxEpsilonIneqDelay'][flow]['A1']))
plot_box_plot(data, 'delay', check, results_dir, args_dir, [int(x * 4500) for x in rateScales], "Success Rate (%)")

In [ ]:
check = 'delay'
for queue in queues_names:
    data = [[] for i in range(len(rateScales))]
    for i in range(len(rateScales)):
        data[i] = results[rateScales[i]][queue + 'Delaystd']
 
    plot_box_plot(data, 'delayStd_' + queue, check, results_dir, args_dir, [int(x * 4500) for x in rateScales], "Delay Std (ns)")

check = 'delay'
for queue in queues_names:
    data = [[] for i in range(len(rateScales))]
    for i in range(len(rateScales)):
        data[i] = results[rateScales[i]][queue + 'DelayMean']
 
    plot_box_plot(data, 'delayMean_' + queue, check, results_dir, args_dir, [int(x * 4500) for x in rateScales], "Delay Mean (ns)")

In [31]:
check = 'delay'
for flow in flows:
    data = [[] for i in range(len(rateScales))]
    for i in range(len(rateScales)):
        data[i] = [x * 100 for x in results[rateScales[i]]['errors'][flow]['A0'] + results[rateScales[i]]['errors'][flow]['A1']]
 
    plot_box_plot(data, 'delayRelativeError', check, results_dir, args_dir, [int(x * 4500) for x in rateScales], "Delay Mean Relative Error(%)")

check = 'delay'
for flow in flows:
    data = [[] for i in range(len(rateScales))]
    for i in range(len(rateScales)):
        data[i] = [x * 100 for x in results[rateScales[i]]['maxEpsilonDelay'][flow]['A0'] + results[rateScales[i]]['maxEpsilonDelay'][flow]['A1']]
 
    plot_box_plot(data, 'delayMaxEpsilon', check, results_dir, args_dir, [int(x * 4500) for x in rateScales], "Delay Mean Maximum Error(%)")